In [19]:
import selenium

In [ ]:
from selenium.common.exceptions import NoSuchElementException   
from selenium import webdriver                                  
from selenium.webdriver.common.by import By                     
from selenium.webdriver.support.ui import WebDriverWait        
import time                                                     
import pandas as pd

# Initialize driver
driver=webdriver.Chrome() 
driver.get("https://www.redbus.in")
driver.maximize_window()  
time.sleep(5)
driver.execute_script("window.scrollBy(0,700)")
#Scraping State Names
scrap=driver.find_elements(By.XPATH,"//div[@class='rtcName']") 
state=[]
for a in scrap: 
     b=a.text
     state.append(b)
     print(b)
#Create a DataFrame and Replace specific state names with their route links
df = pd.DataFrame(state, columns=['state']) # Create DataFrame from state names
df['state'] = df['state'].replace({
    'KERALA RTC': 'ksrtc-kerala',
    'CTU RTC': 'chandigarh-transport-undertaking-ctu',
    'KAAC Transport': 'kaac-transport',
    'TGSRTC': 'tsrtc',
    'SBSTC': 'south-bengal-state-transport-corporation-sbstc',
    'WBTC': 'wbtc-ctc',
    'BSRTC': 'bihar-state-road-transport-corporation-bsrtc',
    'WBSTC': 'west-bengal-transport-corporation'
})
#Initialize empty lists
Route_Link_List = []  # Stores links to each state's route page
datas = []  # Stores details about each bus route
data = []  # Stores each state and its bus route link

#Navigating to Each State's Route URL and Extracting Links
for c in df['state']:
    driver.get(f"https://www.redbus.in/online-booking/{c}")
    #Scrape all elements with the class route, which represent bus routes
    bus_routes=driver.find_elements(By.XPATH,"//a[@class='route']")
    for d in bus_routes:
        print(d.text)
        link = d.get_attribute("href") # get_attribute is a method that retrieves the value of the href,href attribute usually contains the URL (link)
        Route_Link_List.append(link)  # Add the link to the list
        df1 = pd.DataFrame(Route_Link_List, columns=["RouteLink"])
        data.append({
                    "StateName":c,
                    "BusRoute":d.text,
                    "ROUTE LINK":link
                })


time.sleep(15)


# Scroll and fetch elements      
def scroll():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
    # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        
        # Wait for new content to load
       
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break  # If the scroll height doesn't change, we reached the end of the page
        
        last_height = new_height

# Loop through the links in Route_Link_List
for l in Route_Link_List:
    driver.get(l)
    time.sleep(10)
    
    # Try to find and click the button
    try:
        button = driver.find_element(By.XPATH, "(//div[@class='button'])[1]")
        if button.is_displayed():
            button.click()
            time.sleep(3)
            scroll()
    except NoSuchElementException:
        print("No button found, proceeding with the next steps.")

    # Scroll the page if no button is found or after clicking
    scroll()
    
    # Fetch bus names and types again after new content is loaded
    bus_routes = driver.find_element(By.XPATH, "//h1[@class='D136_h1']").text
    bus_name = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dep_times = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']") 
    durations = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    reach_times = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    ratings = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
    prices = driver.find_elements(By.XPATH, "//div[@class='fare d-block']")
    seats_left = driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-16']")
    
    # Extract and store the text from each element
    for bus, btype, dep, duration, reach, rate, price, seat in zip(bus_name, bus_type, dep_times, durations, reach_times, ratings, prices, seats_left):
        bus_names = bus.text
        bus_types = btype.text
        departure_times = dep.text
        durations_list = duration.text
        reach_times_list = reach.text
        ratings_list = rate.text
        prices_list = price.text
        seats_list = seat.text
        
        # Store the details in the list
        datas.append({
            "ROUTE NAMES": bus_routes,
            "ROUTE LINK": l,
            "BUS NAMES": bus_names,
            "BUS TYPES": bus_types,
            "DEPARTURE TIMES": departure_times,
            "DURATIONS": durations_list,
            "REACHING TIME": reach_times_list,
            "RATINGS": ratings_list,
            "PRICES": prices_list,
            "SEATS": seats_list
        })

    # Convert the collected data into a DataFrame
    result = pd.DataFrame(datas)
    df_data = pd.DataFrame(data)

final_df = pd.merge(df_data, result, on="ROUTE LINK", how='inner')

final_df 
final_df.to_csv(r'D:\New folder\redbus_scrap.csv')
df=pd.read_csv(r"D:\New folder\redbus_scrap.csv")

In [ ]:
#Splitting the Route Names
df[['From_Place', 'To_Place']] =df['ROUTE NAMES'].str.split(' to ', expand=True)
df['To_Place']=df['To_Place'].str.replace('Bus','').str.strip()
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df

,STATE NAME,ROUTE NAMES,ROUTE LINK,BUS NAMES,BUS TYPES,DEPARTURE TIMES,DURATIONS,REACHING TIME,RATINGS,PRICES,SEATS,From_Place,To_Place
0,APSRTC,Hyderabad to Vijayawada Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35188,VENNELA (A.C. SLEEPER),00:40,05h 45m,06:25,4.8,737.0,17,Hyderabad,Vijayawada
1,APSRTC,Hyderabad to Vijayawada Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3563,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",01:00,06h 15m,07:15,3.6,412.0,32,Hyderabad,Vijayawada
2,APSRTC,Hyderabad to Vijayawada Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35266,VENNELA (A.C. SLEEPER),01:25,05h 45m,07:10,3.6,737.0,23,Hyderabad,Vijayawada
3,APSRTC,Hyderabad to Vijayawada Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3058,VENNELA (A.C. SLEEPER),02:10,05h 45m,07:55,3.6,737.0,21,Hyderabad,Vijayawada
4,APSRTC,Hyderabad to Vijayawada Bus,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3548,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",03:00,06h 20m,09:20,4.5,412.0,34,Hyderabad,Vijayawada
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5829,JKSRTC,Kishtwar to Jammu Bus,https://www.redbus.in/bus-tickets/kishtwar-to-...,Manpreet Travels,NON A/C Seater (2+2),08:50,08h 40m,17:30,4.0,400.0,27,Kishtwar,Jammu
5830,JKSRTC,Kishtwar to Jammu Bus,https://www.redbus.in/bus-tickets/kishtwar-to-...,Manpreet Travels,NON A/C Seater (2+2),04:00,08h 00m,12:00,2.4,400.0,27,Kishtwar,Jammu
5831,JKSRTC,Kishtwar to Jammu Bus,https://www.redbus.in/bus-tickets/kishtwar-to-...,Manpreet Travels,NON A/C Seater (2+2),05:40,07h 50m,13:30,2.4,400.0,27,Kishtwar,Jammu
5832,JKSRTC,Kishtwar to Jammu Bus,https://www.redbus.in/bus-tickets/kishtwar-to-...,Manpreet Travels,NON A/C Seater (2+2),07:00,08h 00m,15:00,2.4,400.0,27,Kishtwar,Jammu


In [ ]:
#Establishing a Connection to MySQL
import mysql.connector
mydb=mysql.connector.connect(
    host="localhost",
    user="root",
    password="Abinesh",
    auth_plugin="mysql_native_password"
)
mycursor=mydb.cursor()
mycursor.execute("CREATE DATABASE redbus_project")
mycursor.execute("use redbus_project")

In [ ]:
#Creating a Table
mycursor.execute("""
    CREATE TABLE IF NOT EXISTS bus_data (
        state_names TEXT,
        route_names TEXT,                        
        route_link TEXT,
        bus_names VARCHAR(255),
        bus_types VARCHAR(255),
        departure_times VARCHAR(20),
        durations VARCHAR(20),
        reaching_time VARCHAR(20),
        ratings FLOAT,
        prices VARCHAR(20),
        seats VARCHAR(50),
        From_Place VARCHAR(255),
        To_Place  VARCHAR(255)
    )
""")

In [15]:
mycursor.execute("SHOW TABLES")
for i in mycursor:
    print(i)

('bus_data',)


In [ ]:
#inserting into table
inserts="""
INSERT INTO bus_data (
    state_names,route_names,route_link, bus_names, bus_types, 
    departure_times, durations, reaching_time, ratings, 
    prices, seats,From_Place,To_Place
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s,%s,%s)
"""


In [ ]:
#Looping Through DataFrame Rows
for index, row in df.iterrows():
    mycursor.execute(inserts, (
        row['STATE NAME'],
        row['ROUTE NAMES'],
        row['ROUTE LINK'], 
        row['BUS NAMES'], 
        row['BUS TYPES'],
        row['DEPARTURE TIMES'], 
        row['DURATIONS'], 
        row['REACHING TIME'], 
        float(row['RATINGS']), 
        row['PRICES'], 
        row['SEATS'],
        row['From_Place'],
        row['To_Place']
    ))

mydb.commit()
mycursor.close()
mydb.close()

print("Data inserted successfully into the MySQL database!")

Data inserted successfully into the MySQL database!
